In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, when

# Запуск Spark-сессии
spark = SparkSession.builder \
    .appName("Basic PySpark Example") \
    .getOrCreate()

# Примерный датасет сотрудников
data = [
    (1, "Alice", "HR", 4000),
    (2, "Bob", "IT", 5000),
    (3, "Charlie", "IT", 5500),
    (4, "Diana", "Marketing", 4300),
    (5, "Evan", "HR", 4100),
    (6, "Fay", "IT", 6200),
    (7, "Grace", "Marketing", 3800),
    (8, "Henry", "IT", 5000)
]

columns = ["id", "name", "department", "salary"]

df = spark.createDataFrame(data, columns)

# Вывод исходного датафрейма
df.show()

Error: LinkageError occurred while loading main class org.apache.spark.launcher.Main
	java.lang.UnsupportedClassVersionError: org/apache/spark/launcher/Main has been compiled by a more recent version of the Java Runtime (class file version 61.0), this version of the Java Runtime only recognizes class file versions up to 55.0
/home/ivan/project/pyspark-learning/venv/lib/python3.12/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
# Filter (узкая операция)
df_filtered = df.filter(col("salary") > 4500)

# Select и withColumn
df_with_bonus = df.withColumn("salary_with_bonus", col("salary") * 1.1).select("name", "salary_with_bonus")

In [ ]:
# GroupBy + Aggregation (shuffle)
avg_salary_by_dept = df.groupBy("department").agg(avg("salary").alias("avg_salary"))

# Join (shuffle, если по разным partition)
departments = [
    ("HR", "Москва"),
    ("IT", "Санкт-Петербург"),
    ("Marketing", "Казань")
]
dept_df = spark.createDataFrame(departments, ["department", "city"])

df_joined = df.join(dept_df, on="department", how="inner")

# Distinct (shuffle)
distinct_departments = df.select("department").distinct()

# Sort (shuffle)
sorted_df = df.sort(col("salary").desc())

In [ ]:
# Примеры actions:
df_filtered.show()
df_with_bonus.show()
avg_salary_by_dept.show()
df_joined.show()
distinct_departments.show()
sorted_df.show()

# Подсчёт количества сотрудников в каждом отделе
df.groupBy("department").agg(count("*").alias("num_employees")).show()